In [1]:
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
#import matplotlib.pyplot as plt; plt.ion()
from pinocchio.visualize import GepettoVisualizer
from utils.meshcat_viewer_wrapper import  MeshcatVisualizer, colors

In [2]:
robot = robex.load('talos')
model = robot.model
cmodel = cpin.Model(robot.model)
data = model.createData()
cdata = cmodel.createData()

In [3]:
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [4]:
viz.display(robot.q0)

In [9]:
nq = cmodel.nq
nv = cmodel.nv
nx = nq + nv
ndx = nv *2

cx = casadi.SX.sym('cx', nx, 1)
cdx = casadi.SX.sym('cx', ndx, 1)
ca = casadi.SX.sym('ca', nv, 1)

IDX_LF = IDX_LF = cmodel.getFrameId('leg_left_6_link')
IDX_RF = IDX_RF = cmodel.getFrameId('leg_right_6_link')

integrate = casadi.Function('integrate', [cx, cdx], [cpin.integrate(cmodel, cx[: nq], cdx[: nv])] )
com_position = casadi.Function('com', [cx], [cpin.centerOfMass(cmodel, cdata, cx[: nq])] )

cpin.forwardKinematics(cmodel, cdata, cx[: nq], cx[nq :], ca)
cpin.updateFramePlacements(cmodel, cdata)        
lf_position = casadi.Function('lf', [cx], [cdata.oMf[IDX_LF].translation])
rf_position = casadi.Function('rf', [cx], [cdata.oMf[IDX_RF].translation])
rf_rotation = casadi.Function('rf', [cx], [cdata.oMf[IDX_RF].rotation])
get_T_rf = casadi.Function('T_rf', [cx], [cpin.log6(cpin.SE3(rf_rotation(cx), rf_position(cx) ) ).vector])


In [37]:
q0 = robot.q0
v0 = np.zeros(robot.nv)
a0 = np.zeros(robot.nv)
x0 = np.concatenate([q0, v0])

In [38]:
opti = casadi.Opti()

dxs = opti.variable(ndx)
xs = integrate(x0, dxs)

cost = casadi.sumsqr(dxs)

opti.minimize(cost)

#opti.subject_to(com_position(xs[2]) == 0.8)


In [39]:
opti.solver('ipopt')

opti.solve()

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       76

Total number of variables............................:       76
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

OptiSol(Opti {
  instance #9
  #variables: 1 (nx = 76)
  #parameters: 0 (np = 0)
  #constraints: 0 (ng = 0)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [49]:
x_sol = integrate(x0, opti.value(dxs)).full()
q_sol = x_sol[: nq]
v_sol = x_sol[nq:]

In [51]:
viz.display(q_sol)